In [ ]:
# pip install category_encoders

In [ ]:
import pandas as pd
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder
from sklearn.linear_model import Ridge

Escreva uma função de "limpeza" que recebe o nome de um arquivo CSV como entrada e retorna um DataFrame.

A função deve realizar os seguintes passos:

- Selecione os dados no arquivo CSV e retorne apenas apartamentos na Cidade do México ("Distrito Federal") que custem menos de $100.000.

- Remova outliers cortando os 10% inferiores e superiores das propriedades em termos de "surface_covered_in_m2".

- Crie colunas separadas "lat" e "lon".

- A Cidade do México é dividida em 15 bairros. Crie uma feature "borough" a partir da coluna "place_with_parent_names".

- Exclua colunas que contenham mais de 50% de valores nulos.

- Exclua colunas que contenham valores categóricos de baixa ou alta cardinalidade.

- Exclua quaisquer colunas que constituam vazamento para o alvo "price_aprox_usd".

- Exclua quaisquer colunas que criariam problemas de multicolinearidade

In [ ]:
def limpeza(arquivo):
  df = pd.read_csv(arquivo)

  # Mascaras de apt, distrito federal e preco
  mask_apt = df["property_type"] == "apartment"
  mask_ba = df["place_with_parent_names"].str.contains("Distrito Federal")
  mask_price = df["price_aprox_usd"] < 100_000
  df = df[mask_apt & mask_ba & mask_price]

  # Outliers
  low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
  mask_area = df["surface_covered_in_m2"].between(low, high)
  df = df[mask_area]

  # Lat e lon
  df[["lat","lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
  df.drop(columns=["lat-lon"], inplace=True)

  # Borough
  df["borough"] = df["place_with_parent_names"].str.split("|", expand=True)[1]
  df.drop(columns="place_with_parent_names", inplace=True)

  # Mais de 50% vazio
  df.drop(columns=["surface_total_in_m2", "price_usd_per_m2", "floor", "rooms", "expenses"], inplace=True)

  # Cardinalidade
  df.drop(columns=["operation", "property_type", "currency", "properati_url"], inplace=True)

  # Vazamento
  df.drop(columns=["price", "price_aprox_local_currency", "price_per_m2"], inplace=True)

  return df


Use o glob para criar a lista de arquivos. Ela deve conter os nomes dos arquivos de todos os CSVs de imóveis da Cidade do México no diretório ./data, exceto o mexico-city-features.csv.

In [ ]:
files = glob('/content/drive/MyDrive/Buenos-Aires-2/mexico-city-?.csv')

Combine sua função de "limpeza", uma compreensão de lista e pd.concat para criar um DataFrame df.

Ele deve conter todas as propriedades dos cinco arquivos CSV em files.

In [ ]:
frames = []
for file in files:
    df = limpeza(file)
    frames.append(df)

df = pd.concat(frames, ignore_index=True)
print(df.info())
# df.head()

Crie um histograma mostrando a distribuição dos preços de apartamentos ("price_aprox_usd") em df.

Certifique-se de rotular o eixo x como "Price [$]", o eixo y como "Count" e dar o título "Distribution of Apartment Prices". Use o Matplotlib (plt).

Como é a distribuição dos preços? Os dados são normais, ligeiramente assimétricos ou muito assimétricos?

In [ ]:
plt.hist(df["price_aprox_usd"])
plt.xlabel("Price [$]")
plt.ylabel("Count")
plt.title("Distribution of Apartment Prices");

Crie um gráfico de dispersão que mostre o preço dos apartamentos ("price_aprox_usd") como uma função do tamanho dos apartamentos ("surface_covered_in_m2").

Certifique-se de rotular o eixo x como "Area [sq meters]" e o eixo y como "Price [USD]". Seu gráfico deve ter o título "Mexico City: Price vs. Area".

Use o Matplotlib (plt).

In [ ]:
plt.scatter(x=df["surface_covered_in_m2"], y=df["price_aprox_usd"])
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title("Mexico City: Price vs. Area");

Você vê alguma relação entre preço e área nos dados? Como isso é semelhante ou diferente do conjunto de dados de Buenos Aires?

Crie um gráfico de dispersão no Mapbox que mostre a localização dos apartamentos em seu conjunto de dados e represente seu preço usando cores.

Quais áreas da cidade parecem ter preços de imóveis mais altos?

In [ ]:
fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    width = 600,
    height = 600,
    color="price_aprox_usd",
    hover_data=["price_aprox_usd"])

fig.update_layout(mapbox_style="open-street-map")
fig.show()

Crie sua matriz de características X_train e o vetor alvo y_train.

Seu alvo é "price_aprox_usd". Suas características devem ser todas as colunas que restam no DataFrame que você limpou acima.

In [ ]:
features = ["surface_covered_in_m2", "lat", "lon", "borough"]
target = "price_aprox_usd"

X_train = df[features]
y_train = df[target]

X_train.head()

Calcule o erro absoluto médio de referência para o seu modelo.

In [ ]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)
mae_baseline.round(2)

Crie um pipeline chamado model que contenha todos os transformadores necessários para este conjunto de dados e um dos preditores que você usou durante este projeto. Em seguida, ajuste seu modelo aos dados de treinamento.

In [ ]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    Ridge()
)

model.fit(X_train, y_train)

Leia o arquivo CSV mexico-city-features.csv para o DataFrame X_test.

In [ ]:
X_test = pd.read_csv("/content/drive/MyDrive/Buenos-Aires-2/mexico-city-features.csv")
X_test.info()

Use seu modelo para gerar uma Série de previsões para X_test.


In [ ]:
y_test_pred = pd.Series(model.predict(X_test))
y_test_pred.head()

Crie um `Serie` chamado feat_imp. O índice deve conter os nomes de todas as características que seu modelo considera ao fazer previsões;

Os valores devem ser os valores dos coeficientes associados a cada característica.

A Série deve ser ordenada em ordem crescente pelo valor absoluto.

In [ ]:
coef = model.named_steps["ridge"].coef_
feats = model.named_steps["onehotencoder"].get_feature_names_out()

feat_imp = pd.Series(coef, index=feats).sort_values()
feat_imp

Crie um gráfico de barras horizontal que mostre os 10 coeficientes mais influentes para seu modelo.

Certifique-se de rotular o eixo x como "Importance [USD]" e o eixo y como "Feature", e dê ao seu gráfico o título "Feature Importances for Apartment Price". Use o pandas.

In [ ]:
feat_imp.sort_values(key=abs).tail(10).plot(kind='barh')

plt.xlabel('Importance [USD]')
plt.ylabel('Feature')
plt.title('Feature Importances for Apartment Price')